# Project 4  Wrangle OpenStreetMap Data
I choose to use Chicago, IL as the city to study because I am living here right now. I download the data from below website:
- https://www.openstreetmap.org/relation/122604#map=11/41.8340/-87.7324
- https://mapzen.com/data/metro-extracts/metro/chicago_illinois/

## Problems Encountered in the Map
After extracting a small sample size of Chicago data and auditing through it, I have found out several problems in the data:
- Some of the street names and street types are in abbreviations. For example, "Gregory M Sears Dr" should be converted to "Gregory M Sears Drive".
- The sample includes some data not in Chicago, IL. For example, data point with county of "Lake, IN" should not be included into the database.
- The Tiger data is recorded in a different format from the regular data. For example:
```xml
<tag k="name" v="Pinehurst Drive" />
<tag k="highway" v="residential" />
<tag k="tiger:cfcc" v="A41" />
<tag k="tiger:county" v="Kane, IL" />
<tag k="tiger:reviewed" v="no" />
<tag k="tiger:zip_left" v="60506" />
<tag k="tiger:name_base" v="Pinehurst" />
<tag k="tiger:name_type" v="Dr" />
<tag k="tiger:zip_right" v="60506" />```

### Street Names and Street Types
I use `audit_street_name.py` to audit the street names. From the sample I find out some street names that need to be updated to full street types. For example, Ave to Avenue, Dr to Drive. I use the mapping method in the case study and the `update_st_name` function to update the street names.

### Postal Codes
I use `audit_postal_code.py` to audit the postal codes. I build up the `audit_pc` function to map up the city/county and postal code of each data point. In regular data points, the postal codes are all correct - they are within the Great Chicago area. But among the Tiger data points which are collected from GPS, some of the data are in IN instead of IL. So I create a function `is_not_in` to figure out whether the data point from osm file is in IN state. If so, I will not put it into the csv files.

### Tiger Data
There are a lot of data pulled from Tiger GPS. I use `audit_tiger.py` to audit these data because they have different values in second level "k" tags from the regular data. I audit the street names and it seems no need to update it in the sample data - there are no abbreviations. But I will still update the street names for Tiger data in cleaning process for larger size of data. I also audit the postal code of Tiger data, and as I mentioned above, it includes some IN data. I will eliminate them in the cleaning process.

## Overview of the Data
After using `clean_convert.py` to clean and convert the xml file into csv file, I create the database from csv file and use sql queries to gather the data overview.

### File Size
- Original uncompressed osm file: chicago_illinois.osm ----- 2.08GB
- Sample osm file: sample.osm ----- 2.1MB
- Database: chicago.db ----- 2.4MB

### Number of Nodes and Ways
```sql
sqlite> SELECT COUNT(*) FROM nodes;
8708```

```sql
sqlite> SELECT COUNT(*) FROM ways;
1221```

### Top 10 Contributing Users
```sql
sqlite> SELECT user, COUNT(*) as count
   ...> FROM
   ...> (SELECT user FROM ways UNION ALL
   ...> SELECT user FROM nodes) u
   ...> GROUP BY user
   ...> ORDER BY count DESC
   ...> LIMIT 10;
   
chicago-buildings,5609
Umbugbene,1112
woodpeck_fixbot,223
alexrudd (NHD),201
g246020,117
asdf1234,107
patester24,104
Oak_Park_IL,102
mpinnau,96
TIGERcnl,88```

### Number of Unique Users
```sql
sqlite> SELECT COUNT(DISTINCT(u.uid))
   ...> FROM
   ...> (SELECT uid FROM ways UNION ALL
   ...> SELECT uid FROM nodes) u;
376```

### Number of Users Contributing Only Once
```sql
sqlite> SELECT COUNT(*)
   ...> FROM
   ...> (SELECT n.uid, COUNT(*) as count
   ...> FROM (SELECT uid FROM ways UNION ALL
   ...> SELECT uid FROM nodes) n
   ...> GROUP BY n.uid
   ...> HAVING count=1) m;
   
178```

### Other Tops in Tags
- Top recorded points at highway:
```sql
sqlite> SELECT n.value, COUNT(*) as count
   ...> FROM 
   ...> (SELECT * FROM ways_tags UNION ALL
   ...> SELECT * FROM nodes_tags) n
   ...> WHERE key='highway'
   ...> GROUP BY n.value
   ...> ORDER BY count DESC
   ...> LIMIT 5;
residential,104
service,70
footway,17
secondary,15
bus_stop,14```

- Top amenities:
```sql
sqlite> SELECT n.value, COUNT(*) as count
   ...> FROM
   ...> (SELECT * FROM ways_tags UNION ALL
   ...> SELECT * FROM nodes_tags) n
   ...> WHERE key='amenity'
   ...> GROUP BY n.value
   ...> ORDER BY count DESC
   ...> LIMIT 5;
parking,18
school,5
place_of_worship,2
restaurant,2
bicycle_repair_station,1```

## Other Ideas about the Datasets

### Contribution Rate
Combining nodes and ways, the number of records in the sample is 9,929. Excluding the user *chicago-building* (I believe that it is the administator to mark down the main buildings in Chicago), the number of contributing points is 4,320. Among these points, user *Umbugbene* contributes 25.74% of data, and top 10 users contributes over half of the data. 47.34% of users contributes only once to the dataset, and they contributes only 4.12% in total. Apparently the Chicago data might not be robust if one of the influential users keep reporting incorrect data points. 

### City and Postal Code
From the data auditing and cleaning process, it is easy to tell that the data points do not include all information about the node. For example, not all points contain city name, county name, or zip code. I query for top cities and top postal cods as below:

- Top cities:
```sql
sqlite> SELECT value, COUNT(*) as count
   ...> FROM
   ...> (SELECT * FROM ways_tags UNION ALL
   ...> SELECT * FROM nodes_tags) n
   ...> WHERE key LIKE '%city'
   ...> GROUP BY value
   ...> ORDER BY count DESC
   ...> LIMIT 5;
Evanston,16
Chicago,4
Naperville,4
River Forest,4
Downers Grove,2```

- Top postal codes:
```sql
sqlite> SELECT n.value, COUNT(*) as count
   ...> FROM
   ...> (SELECT * FROM ways_tags UNION ALL
   ...> SELECT * FROM nodes_tags) n
   ...> WHERE n.key='postcode'
   ...> OR n.key LIKE 'zip%'
   ...> GROUP BY n.value
   ...> ORDER BY count DESC
   ...> LIMIT 5;
60201,8
60202,8
60004,6
60477,6
60515,5```

The top 2 postal codes are for Evanston, which is consistent with the result of top cities. But the following zip codes are for Arlington Heights and Tinley Park that are not in top cities. It is just a simple query to see the inconsistency between city and postal code. By doing this, we can tell that the dataset is incomplete and inconsistent.

### Solutions
The above problems of the OpenStreetMap data could be solved by relying more on GPS data. By introducing the GPS technology, the record of map data will be more complete and accurate. Instead of manually reporting the map data by users, GPS data is more robust and unbiased, especially when users are not familiar with the ways. It will also improve the dataset if specific framework and standard is introduced into the GPS data collection process. However, it is not easy to have access to as much as GPS data on private cell phones and cars - it might raise privacy issue about personal activities. In addition, the interval between each update should be set properly. Using GPS to update the map data might hugely increase the size of the dataset by adding a lot of repeated information to one specific node.

## References
- [OpenStreetMap Data Case Study](https://gist.github.com/carlward/54ec1c91b62a5f911c42#file-sample_project-md)
- [OpenStreetMap Sample Project Data Wrangling with MongoDB](https://docs.google.com/document/d/1F0Vs14oNEs2idFJR3C_OPxwS6L0HPliOii-QpbmrMo4/pub)
- [SQL Schema](https://gist.github.com/swwelch/f1144229848b407e0a5d13fcb7fbbd6f)